In [1]:
#加载模型太慢了。。。单独拿出来

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer
from gensim.models import Word2Vec
import re
import gensim


# path = "/home/user/Programs/PMA_lihang/mypaper/module/bert_large_uncased"
# #加载bert模型
# model = BertModel.from_pretrained(path)
# #加载分词器
# tokenizer = BertTokenizer.from_pretrained(path)

# for param in model.parameters():
#     param.requires_grad = False
 
model = gensim.models.KeyedVectors.load_word2vec_format("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin", binary=True)
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/word2vec.model")
#加载谷歌新闻预训练的word2vec模型
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin")
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin", binary=True)


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer
from gensim.models import Word2Vec
import re
import gensim


# path = "/home/user/Programs/PMA_lihang/mypaper/module/bert-base-uncased"
# #加载bert模型
# model = BertModel.from_pretrained(path)
# #加载分词器
# tokenizer = BertTokenizer.from_pretrained(path)

# for param in model.parameters():
#     param.requires_grad = False
 
# model = gensim.models.KeyedVectors.load_word2vec_format("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin", binary=True)
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/word2vec.model")
#加载谷歌新闻预训练的word2vec模型
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin")
# model = Word2Vec.load("/home/user/Programs/PMA_lihang/code_lihang/word2vec/model/GoogleNews-vectors-negative300.bin", binary=True)
from gensim.parsing.preprocessing import STOPWORDS 
stop_words = STOPWORDS


def deal_with_text(v_type):
    v_type_ = []
    for i in range(len(v_type)):
        content = v_type[i]
        content = re.sub("[^\u4e00-\u9fa5^a-z^A-Z^0-9]", " ", str(content))
        # 只保留数字和字母
        content = re.sub("[^a-z^A-Z^0-9]", " ", str(content))
        #去除连续的空格
        content = re.sub("\s+", " ", content)
        #删除回车符
        content = re.sub("\n", " ", content)
        #删除开头和结尾的空格
        content = content.strip()
        #print(content)
        content = content.split(" ")
        #将所有大写字母转换为小写
        content = [word.lower() for word in content]
        for j in range(len(content)):
            v_type_.append(content[j])
            # print("处理后的文本：")
            # print(content[j])
        # print(v_type_)

         
        # 定义一个文本字符串 
        # text = "He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and fishery rihgts at once." 
        # # 对文本进行分词（默认使用空格分隔）
        # tokens = text.split()
        # # 遍历分词结果，删除在停用词列表中的词 
        # filtered_tokens = [token for token in v_type_ if token not in stop_words] 
        # # 将过滤后的分词结果重新拼接成字符串 
        
        # filtered_text = ' '.join(filtered_tokens)
        # v_type_ = filtered_text.split(" ")
        # print(v_type_)
    return v_type_



def get_sentence_embedding(text, text2):
    # tokenize text and convert tokens to ids
    tokens = tokenizer.tokenize(text) # ['i', 'like', 'strawberries']
    print(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens) # [1045, 2066, 13137]

    tokens2 = tokenizer.tokenize(text2) # ['i', 'like', 'strawberries']
    print(tokens2)
    input_ids2 = tokenizer.convert_tokens_to_ids(tokens2) # [1045, 2066, 13137]
    
    # print(input_ids)
    # print(input_ids2)

    # add special tokens [CLS] and [SEP]
    input_ids = [tokenizer.cls_token_id] + input_ids + [tokenizer.sep_token_id] + input_ids2 + [tokenizer.sep_token_id] # [101, 1045, 2066, 13137, 102]
        # [101, 1045, 2066, 13137, 102]

    # convert input ids to tensor
    input_tensor = torch.tensor([input_ids]) # shape (1,5)

    # print(input_tensor.shape)   
    # feed input tensor to BERT model
    with torch.no_grad():
        output = model(input_tensor)

    
    # get the last hidden states of BERT
    last_hidden_state = output.last_hidden_state # shape (1,5,dim)
    # print(last_hidden_state.shape)
    # get the sentence embedding by taking the first token ([CLS])
    sentence_embedding = last_hidden_state#[:,0,:] # shape (1,dim)
    # print(sentence_embedding.shape)
    return sentence_embedding



# for i in range(len(v_type_description) - 1):
#     print(v_type_description[i])

#计算两个tensor的余弦相似度，shape是(1,768)
def cos_sim(a, b):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    cos_sim = cos(a, b)
    return cos_sim

#计算两个tensor的欧式距离，shape是(1,768)
def euclidean_dist(a, b):
    dist = nn.PairwiseDistance(p=2)
    dist = dist(a, b)
    return dist

#计算两个tensor的曼哈顿距离，shape是(1,768)
def manhattan_dist(a, b):
    dist = nn.PairwiseDistance(p=1)
    dist = dist(a, b)
    return dist

#定义
# a = get_sentence_embedding(text)
# for i in v_type_description:
#     b = get_sentence_embedding(i)
#     print(cos_sim(a, b))

# a = get_sentence_embedding(text)
# for i in v_type_description:
#     b = get_sentence_embedding(i)
#     print(euclidean_dist(a, b))

# a = get_sentence_embedding(text)
# for i in v_type_description:
#     b = get_sentence_embedding(i)
#     print(manhattan_dist(a, b))

#自己训练的word2vec
def get_vectorOOV1(s):
  try:
    return np.array(model.wv.get_vector(s))
  except KeyError:
    #print("$$$$$$$")
    return np.random.random((300,))

def get_vector(v_type_):
    v_type_vec=[]
    for i in range(len(v_type_)):
        temp = []
        #for j in range(len(v_type_[i])):
        temp.append(get_vectorOOV1(v_type_[i]))
        # for j in range(10-len(v_type_[i])):
        #     temp.append(np.random.random((300,)))
        v_type_vec.append(temp)
    return v_type_vec

#新闻word2vec
def get_vectorOOV2(s):
  try:
    return np.array(model.get_vector(s))
  except KeyError:
    #print("$$$$$$$")
    return np.random.random((300,))

def get_vector2(v_type_):
    v_type_vec=[]
    for i in range(len(v_type_)):
        temp = []
        #for j in range(len(v_type_[i])):
        temp.append(get_vectorOOV2(v_type_[i]))
        # for j in range(10-len(v_type_[i])):
        #     temp.append(np.random.random((300,)))
        v_type_vec.append(temp)
    return v_type_vec

# 导入WordNetLemmatizer和word_tokenize
from nltk.stem import WordNetLemmatizer

# 创建一个WordNetLemmatizer对象
lemma = WordNetLemmatizer()

def stemSentence(words):
    for i in range(len(words)):
        words[i] = lemma.lemmatize(words[i])
    return words



In [38]:


#用word2vec模型 分类 att_type 自身的描述 

from gensim.parsing.preprocessing import STOPWORDS 
import numpy 
stop_words = STOPWORDS 
# 定义一个文本字符串 
def filter(tokens):
    filtered_tokens = [token for token in tokens if token not in stop_words and len(token) > 1] 
    return filtered_tokens
# text = "Buffer overflow Buffer overflow Buffer overflow Buffer overflow Buffer overflow Buffer overflow Buffer overflow"#"Buffer overflow, in Solaris x86 mkcookie allows local users to obtain root access."
# text = "Buffer overflow, in Solaris x86 mkcookie allows local users to obtain root access."
# text1 = "attackers with Overall/Read permission"
# text2 = "The Remote Data Service Object (RDS.DataControl) in Microsoft Internet Explorer 6 on Windows 2000 allows remote attackers to cause a denial of service (crash) via a series of operations that result in an invalid length calculation when using SysAllocStringLen, then triggers a buffer over-read."

# text = "an attacker, A flaw was found in the hivex library. This flaw allows to input a specially crafted Windows Registry (hive) file, which would cause hivex to recursively call the _get_children() function, leading to a stack overflow. The highest threat from this vulnerability is to system availability."
# text = "A  attacker,An out-of-bounds write issue was addressed with improved bounds checking. This issue is fixed in macOS Monterey 12.3. A remote attacker may be able to cause unexpected system termination or corrupt kernel memory."
#text = "A attacker ,A logic issue was addressed with improved state management. This issue is fixed in iTunes 12.12.4 for Windows. A attacker may be able to elevate their privileges."
# tetx = "an attacker, ASP.NET Core 2.0 allows to steal log-in session information such as cookies or authentication tokens via a specially crafted URL aka ""ASP.NET Core Elevation Of Privilege Vulnerability""."

# text = "via a UDP packet, resulting in Denial of Service.handle_ipDefaultTTL in agent/mibgroup/ip-mib/ip_scalars.c in Net-SNMP 5.8 through 5.9.3 has a NULL Pointer Exception bug that can be used by a remote attacker (who has write access) to cause the instance to crash via a crafted UDP packet, resulting in Denial of Service."
def get_similar_score(text1,text2):
    # print("函数中的")
    # print(text1)
    # print(text2)
    text = text1.split(" ")

    # print(text1)
    text = deal_with_text(text)
    text = filter(text)
    text = stemSentence(text)
    # text1[0] = "remote"
    # print(text1)
    aspect = get_vector2(text)
    #list转tensor
    # print(len(a[0][0]))
    aspect = torch.tensor(aspect)
    #把第二维度删除
    aspect = torch.squeeze(aspect)
    #如果维度为300，则在第一维度增加一个维度
    if aspect.shape[0] == 300:#只有一个词，则相似度忽略，凭借其他方面判断
        return torch.tensor(numpy.array(0.0))
        # print("#######")
        aspect = aspect.unsqueeze(0)

    # print(aspect.shape)
    #取平均值
    if aspect.shape[0] == 1:
        aspect2 = torch.mean(aspect, 0)#att_type
    # if aspect.shape[0] > 1:
    #     aspect = torch.mean(aspect[:1], 0 )*0.2 + torch.mean(aspect[1:], 0)*0.8
    #     aspect2 = torch.mean(aspect[:1], 0 )*0.8 + torch.mean(aspect[1:], 0)*0.2
    # else:
    #     aspect = torch.mean(aspect, 0)
    #     aspect2 = torch.mean(aspect, 0)
    # print(a.shape)
    else:
    # aspect = torch.mean(aspect[:1], 0 )*0.2 + torch.mean(aspect[1:], 0)*0.8
        aspect2 = torch.mean(aspect[:1], 0 )*0.8 + torch.mean(aspect[1:], 0)*0.2#att_type
        #后半部分
        #middle = int(aspect.shape[0]/2)
        #为了处理execute arbitrary code or cause a denial of service (memory corruption)，后半部分的相似度
        #初始化aspect2_

        flag = False
        if aspect.shape[0] > 5:
            flag = True
            aspect2_ = torch.mean(aspect[3:4], 0 )*0.8 + torch.mean(aspect[4:], 0)*0.2#att_type  
        

    text = text2.split(" ")
    
    text = deal_with_text(text)
    text = filter(text)
    text = stemSentence(text)
    # print(text)
    # text[0] = "remote"
    # print(text)
    # a = get_vector(text)
    label = get_vector2(text)
    
    
    #list转tensor
    label = torch.tensor(label)
    #把第二维度删除
    label = torch.squeeze(label)
    # print(label.shape)
    # print("^^^^^^^^^^^^^^^^^^^^")
    # print(label.shape[0])
    if label.shape[0] == 300:
        # print("********")
        label = label.unsqueeze(0)
    # print(label.shape)
    #取平均值
    if label.shape[0] == 1:
        label2 = torch.mean(label, 0)
    # 取加权平均值，长度的前20%的token有80%的权重，长度的后80%的token有20%的权重
    # b = torch.mean(b[0:int(len(b)*0.2)], 0) * 0.8 + torch.mean(b[int(len(b)*0.2):len(b)], 0) * 0.2
    #
    # if label.shape[0] > 1:
    #     print("here",label.shape[0])
    #     label = torch.mean(label[:1], 0 )*0.2 + torch.mean(label[1:], 0)*0.8
    #     label2 = torch.mean(label[:1], 0 )*0.8 + torch.mean(label[1:], 0)*0.2
    # else:
    #     label = torch.mean(label, 0)
    #     label2 = torch.mean(label, 0)

    # label = torch.mean(label[:1], 0 )*0.2 + torch.mean(label[1:], 0)*0.8
    else:
        label2 = torch.mean(label[:1], 0 )*0.8 + torch.mean(label[1:], 0)*0.2

    # print(b.shape)
    # print(cos_sim(aspect, label))
    # print(euclidean_dist(aspect, label))
    # print(label == aspect)
    # print(manhattan_dist(aspect, label))

    # s1 = cos_sim(aspect, label)
    s2 = cos_sim(aspect2, label2)
    # print(type(s1),type(s2))
    if flag:
        # print("执行比较后半部分")
        s2_ = cos_sim(aspect2_, label2)
        if s2 < s2_:
            return s2_
    # print("s1s2",s1.shape,s2.shape)
    # if s1 > s2:
    #     return s1
    # else:
    #     return s2
    # print("sim:",s1)
    # print(text1,text2)
    # print("描述、标签sim:",s2)
    
    return s2

# path_att_type = "/home/user/Programs/PMA_lihang/mypaper/module/temp.txt"
path_att_type = "/home/user/Programs/PMA_lihang/mypaper/module/temp_impact.txt"
# path_v_type_description = "/home/user/Programs/PMA_lihang/mypaper/label_description/att_type_aspect_context.txt"#v_type.txt"
path_v_type_description = "/home/user/Programs/PMA_lihang/mypaper/label_description/att_impact_aspect.txt"
f_v_type = open(path_v_type_description, 'r', encoding='utf-8')
f_v_type_reader = f_v_type.readlines()#

v_type_description = []#我对标签的扩充描述
# ii = 0

f_att_type = open(path_att_type, 'r', encoding='utf-8')
f_att_type_reader = f_att_type.readlines()
att_type_description = []#数据中的方面描述


ii = 0
count = [] #统计每个类型出现的次数 改成用字典来存 描述和数量

for line in f_att_type_reader:
    # contens = line.strip().split(" ## ")

    att_type_description.append(line.strip().split("\t")[0])

    count.append(int(line.strip().split("\t")[-1]))
    #处理前几个高频出现的类型


    #下次运行，把下面的代码注释，要遍历所有的数据，效果会更好
    ii += 1
    if ii == 400:
        break

# print(v_type_description)

import csv
# path  = "/home/user/Programs/PMA_lihang/mypaper/module/temp1.txt"
path  = "/home/user/Programs/PMA_lihang/mypaper/module/temp1_impact.txt"
f = open(path, 'a', encoding='utf-8')

def compare_first():#
    #只取第一个
    for line in f_v_type_reader: #
        # contens = line.strip().split(" ## ")
        v_type_description.append(line.strip().split("##")[0])
        # ii += 1

    for text1 in att_type_description:
        kk = 1
        
        for i in v_type_description:
            
            #将kk写入txt文件
            f.write(str(kk) + "\n")
            f.write(str(text1) + "\n")
            f.write(str(i) + "\n")

            # print(text1)
            # print(i)

            score = get_similar_score(text1,i)
            #tensor转float
            # score.item()

            f.write(str(score) + "\n")
            kk += 1
        #写入文件
# compare_first()


def compare_all():#比较各个描述与所有标签的相似度

    
    iikk = 0    
    for text1 in att_type_description:#对于一条数据描述
        print(iikk)
        iikk += 1
        if iikk == 20:
            break
        # print(text1)
        

        candidate = []#一个类别的所有描述  
        kk = 0
        for line in f_v_type_reader:#取一行标签扩充的描述
            #第几类

            # print(line)
            #每次的一行只是一个类别
            
            candidate = line.split(" ## ") 
            # candidate = list(filter(candidate))
            candidate = [x for x in candidate if x != '' and x != '\n']
            #定义最小的tensor相似度为-1
            sim = torch.tensor(numpy.array(-1.0))
            f.write("序号"+str(kk) + "\n")
            f.write("文本"+str(text1) + "\n")
            # f.write(str(candidate[0]) + "\n")
            # print("***********")
            # print(candidate)
            #获取最相似的
            ii = 0
            kk += 1

            for text2 in candidate:#对于一个标签的每个描述
                # f.write(str(text2) + "\n")
                # print(i)
                #将kk写入txt文件
                # print("输入的两个文本：")
                # print(type(text1))
                # print(type(text2))
                #取最大的
                
                s = get_similar_score(str(text1),str(text2))
                
                # print("&&&&&&&&&&&&&&",text2.strip() ,"&&&&&&&&&&&&&")
                f.write("标签"+text2.strip() + "\n")
                # f.write("\n")
                f.write("得分"+str(s) +"\n")
                # f.write( "\n")
                # print(type(s))
                # print("得到的相似度：")
                # print(s)
                if sim < s:
                    sim = s
                    jj = ii
                # sim = max(sim, get_similar_score(text1,i).item())
                #score = get_similar_score(text1,i)
                #tensor转float
                # score.item()
                
                ii += 1
            # f.write(str("****最大的项******" + "\n"))
            f.write("**********最大的项************"+"\n")
            f.write(candidate[jj] + str(sim) + "\n")
            # print("sim: ",sim)
            f.write("\n")
            
            # print(candidate)
        f.write("####################################")
print("处理完数据，开始循环")
compare_all()

import time
list_all = []
count_all = []
def get_class_num():#打印各个类别的实际中的描述
    #一共6个类别
    # for i in range(7):
    for i in range(10):
        list_all.append([])
        count_all.append(0)
    #所有的描述
    iii = 0
    for text1 in att_type_description: # 数据中的描述
        time.sleep(0.1)
        # print(text1)
        kk = 1
        #类别index
        # class_num = -1
        class_num = []

        max_sim = -1
        #获取类别
        class_now = 0
        for line in f_v_type_reader:
            # print(line)
            #每次的一行只是一个类别
            candidate = line.split(" ## ")
            # candidate = list(filter(candidate))
            candidate = [x for x in candidate if x != '' and x != '\n']
            #定义最小的tensor相似度为-1
            sim = torch.tensor(numpy.array(-1.0))
            # f.write("序号"+str(kk) + "\n")
            # f.write("文本"+str(text1) + "\n")
            # f.write(str(candidate[0]) + "\n")
            # print("***********")
            # print(candidate)
            #获取最相似的
            ii = 0
            for text2 in candidate:
                # f.write(str(text2) + "\n")
                # print(i)
                #将kk写入txt文件
                # print("输入的两个文本：")
                # print(type(text1))
                # print(type(text2))
                #取最大的
                
                s = get_similar_score(str(text1),str(text2))
                
                # print("&&&&&&&&&&&&&&",text2.strip() ,"&&&&&&&&&&&&&")
                # f.write("标签"+text2.strip() + "\n")
                # f.write("\n")
                # f.write("得分"+str(s) +"\n")
                # f.write( "\n")
                # print(type(s))
                # print("得到的相似度：")
                # print(s)
                if sim < s:
                    sim = s
                    jj = ii
                    
                # sim = max(sim, get_similar_score(text1,i).item())
                #score = get_similar_score(text1,i)
                #tensor转float
                # score.item()
                
                ii += 1
                # 临时加的
                if s > 0.9:
                    class_num.append(class_now)
                    break

            if max_sim < sim:
                max_sim = sim
                #class_num = class_now

            # f.write(str("****最大的项******" + "\n"))
            # f.write("**********最大的项************"+"\n")
            # f.write(candidate[jj] + str(sim) + "\n")
            # print("sim: ",sim)
            # f.write("\n")
            kk += 1
            # print(candidate)

            class_now += 1

        if max_sim < 0.9:
            # class_num = 6
            # class_num = 9
            class_num.append(9)
        
        # list_all[class_num].append(text1)
        # #统计类别的总数
        # count_all[class_num] += count[iii]
        # iii += 1

        # 处理这条数据所属的多个类别
        for i in class_num:
            list_all[i].append(text1)
            #统计类别的总数
            # count_all[i] += count[iii]
            # iii += 1
        


        

#统计每个类别的数量
# path = "/home/user/Programs/PMA_lihang/mypaper/module/temp2.txt"
# path = "/home/user/Programs/PMA_lihang/mypaper/module/temp2_impact.txt"
# f = open(path, 'w', encoding='utf-8')
# get_class_num()
# for i in list_all:
#     for j in i:
#         f.write(j + "\n")
#     f.write("******************\n")
# f.close()
# # count_all[6] += 7500
# for i in count_all:
#     print(i)   
#将分类的情况按照类别写入文件
#建一个列表，每个元素是一个类别，每个类别是一个列表




处理完数据，开始循环
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [36]:
s1 = "execute arbitrary code or cause a denial of service (memory corruption)"
s2 = "cause a denial of service (memory corruption)"

score = get_similar_score(s1,s2)
print(score)

执行比较后半部分
tensor(1.)


In [9]:
path = "/home/user/Programs/PMA_lihang/mypaper/module/temp2_impact.txt"
f = open(path, 'w', encoding='utf-8')
get_class_num()
for i in list_all:
    for j in i:
        f.write(j + "\n")
    f.write("******************\n")
f.close()
# count_all[6] += 7500
for i in count_all:
    print(i)   

78440
10184
11872
1362
3276
584
24752
1174
0
0
722
7580
0
0
0
0
0
0
0
0
0
0
0
0
